In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy.spatial.transform import Rotation as R
import time
import pickle
import argparse 
from utils import *

from deepVCP import DeepVCP
from ModelNet40Dataset import ModelNet40Dataset
from KITTIDataset import KITTIDataset
from deepVCP_loss import deepVCP_loss

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
  

In [2]:
dataset = "modelnet"
retrain_path = False
model_path = "final_model.pt"
full_dataset = True 

In [3]:
num_epochs = 10
batch_size = 1
lr = 0.001
# loss balancing factor 
alpha = 0.5

print(f"Params: epochs: {num_epochs}, batch: {batch_size}, lr: {lr}, alpha: {alpha}\n")

# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

# dataset 

root = 'modelnet40_normal_resampled/'
shape_names = np.loadtxt(root+"modelnet10_shape_names.txt", dtype="str")
train_data= ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset, split='train')
test_data = ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset,  split='test')


train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


num_train = len(train_data)
num_test = len(test_data)
print('Train dataset size: ', num_train)
print('Test dataset size: ', num_test)


Params: epochs: 10, batch: 1, lr: 0.001, alpha: 0.5

device: cuda
# Total clouds 2
# Total clouds 2
Train dataset size:  2
Test dataset size:  2


In [5]:
use_normal = False if dataset == "kitti" else True

# Initialize the model
model = DeepVCP(use_normal=use_normal)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

# Retrain
if retrain_path:
    print("Retrain on ", retrain_path)
    model.load_state_dict(torch.load(retrain_path))
else:
    print("No retrain")

# Define the optimizer
optim = Adam(model.parameters(), lr=lr)

# begin train 
model.train()
loss_epoch_avg = []
for epoch in range(num_epochs):
    print(f"epoch #{epoch}")
    loss_epoch = []
    running_loss = 0.0
    
    for n_batch, (src, target, R_gt, t_gt, ) in enumerate(train_loader):
        start_time = time.time()
        # mini batch
        src, target, R_gt, t_gt = src.to(device), target.to(device), R_gt.to(device), t_gt.to(device)
        t_init = torch.zeros(1, 3)
        src_keypts, target_vcp = model(src, target, R_gt, t_init)
        # print('src_keypts shape', src_keypts.shape)
        # print('target_vcp shape', target_vcp.shape)
        # zero gradient 
        optim.zero_grad()
        loss, R_pred, t_pred = deepVCP_loss(src_keypts, target_vcp, R_gt, t_gt, alpha=0.5)

        # error metric for rigid body transformation
        r_pred = R.from_matrix(R_pred.squeeze(0).cpu().detach().numpy())
        r_pred_arr = torch.tensor(r_pred.as_euler('xyz', degrees=True)).reshape(1, 3)
        r_gt = R.from_matrix(R_gt.squeeze(0).cpu().detach().numpy())
        r_gt_arr = torch.tensor(r_gt.as_euler('xyz', degrees=True)).reshape(1, 3)
        pdist = nn.PairwiseDistance(p = 2)
        
        t_gt = t_gt.reshape(1,3)
        t_pred = t_pred.reshape(1,3)
        print("rotation error: ", pdist(r_pred_arr, r_gt_arr).item())
        print("translation error: ", pdist(t_pred, t_gt).item())

        # backward pass
        loss.backward()
        # update parameters 
        optim.step()

        running_loss += loss.item()
        loss_epoch += [loss.item()]
        print("--- %s seconds ---" % (time.time() - start_time))
        if (n_batch + 1) % 5 == 0:
            print("Epoch: [{}/{}], Batch: {}, Loss: {}".format(
                epoch, num_epochs, n_batch, loss.item()))
            running_loss = 0.0
    
    torch.save(model.state_dict(), "epoch_" + str(epoch) + "_model.pt")
    loss_epoch_avg += [sum(loss_epoch) / len(loss_epoch)]
    with open("training_loss_" + str(epoch) + ".txt", "wb") as fp:   #Pickling
        pickle.dump(loss_epoch, fp)

No retrain
epoch #0
Processing file: bathtub_0001
feature extraction time:  10.875095129013062
src_keypts_idx_unsqueezed:  torch.Size([1, 6, 64])
src_keypts:  torch.Size([1, 64, 6])
Grouping keypoints time:  0.027999401092529297
B:  1
K_topk:  64
nsample:  32
num_feat:  32
get_cat_feat_src time:  0.0
tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([ 406, 5828, 9650,  ..., 4538, 9128, 9993], device='cuda:0')
get_cat_feat_tgt time:  0.1080007553100586
Loss: 0.35361389591598574
rotation error:  0.005719051965166379
translation error:  0.9412155939426217
--- 21.79595446586609 seconds ---
Processing file: bathtub_0002
feature extraction time:  10.740953207015991
src_keypts_idx_unsqueezed:  torch.Size([1, 6, 64])
src_keypts:  torch.Size([1, 64, 6])
Grouping keypoints time:  0.024999380111694336